In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
a = pd.read_csv("../data/raw/RAW_interactions.csv")
a.head(2)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."


In [4]:
b = pd.read_csv("../data/raw/RAW_recipes.csv")
b.head(3)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13


In [5]:
a.shape , b.shape

((1132367, 5), (231637, 12))

In [6]:
a = a.sample(50000)
b = b.sample(50000)

In [7]:
data = pd.merge(a,b, right_on='id',left_on='recipe_id')

In [8]:
data.head(2)

,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,2001560643,2886,2017-05-29,5,Awesome recipe!! Made it twice already..both t...,best banana bread,2886,65,1762,1999-09-26,"['time-to-make', 'course', 'main-ingredient', ...","[272.8, 16.0, 97.0, 14.0, 7.0, 31.0, 14.0]",13,"['remove odd pots and pans from oven', 'prehea...",you'll never need another banana bread recipe ...,"['butter', 'granulated sugar', 'eggs', 'banana...",8
1,2001957150,2886,2018-01-27,5,moist and delicious! I added 1/2 tsp cinnamon ...,best banana bread,2886,65,1762,1999-09-26,"['time-to-make', 'course', 'main-ingredient', ...","[272.8, 16.0, 97.0, 14.0, 7.0, 31.0, 14.0]",13,"['remove odd pots and pans from oven', 'prehea...",you'll never need another banana bread recipe ...,"['butter', 'granulated sugar', 'eggs', 'banana...",8


In [9]:
data.drop(["user_id","submitted","contributor_id","id"],axis=1,inplace=True)

In [26]:
rec = data[["recipe_id","rating","name","tags","description","ingredients",]]

In [27]:
rate = rec.groupby("name")["rating"].sum().reset_index()

In [28]:
rec.duplicated().sum()
rec.drop_duplicates(inplace=True)

<ipython-input-28-cf0ddfdefa66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec.drop_duplicates(inplace=True)


In [29]:
rec.reset_index(drop=True,inplace=True)

In [30]:
rec.head(2)

,recipe_id,rating,name,tags,description,ingredients
0,2886,5,best banana bread,"['time-to-make', 'course', 'main-ingredient', ...",you'll never need another banana bread recipe ...,"['butter', 'granulated sugar', 'eggs', 'banana..."
1,2886,4,best banana bread,"['time-to-make', 'course', 'main-ingredient', ...",you'll never need another banana bread recipe ...,"['butter', 'granulated sugar', 'eggs', 'banana..."


In [31]:
import string

def convert_to_list(x):
    return str(x).strip('[]').replace('"', '').replace("'", '').replace(' ', '').split(',')

In [32]:
rec["tags"] = rec["tags"].apply(lambda x: convert_to_list(x))
rec["ingredients"] = rec["ingredients"].apply(lambda x: convert_to_list(x))
rec["description"] = rec["description"].apply(lambda x: convert_to_list(x))

<ipython-input-32-334920e2948a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec["tags"] = rec["tags"].apply(lambda x: convert_to_list(x))
<ipython-input-32-334920e2948a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec["ingredients"] = rec["ingredients"].apply(lambda x: convert_to_list(x))
<ipython-input-32-334920e2948a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [33]:
rec["rec"] = rec["tags"] + rec["description"] + rec["ingredients"]
rec = rec[~rec.duplicated("name")]
rec.reset_index(drop=True,inplace=True)

<ipython-input-33-b2ddd3082bac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec["rec"] = rec["tags"] + rec["description"] + rec["ingredients"]


In [34]:
rec.head(2)

,recipe_id,rating,name,tags,description,ingredients,rec
0,2886,5,best banana bread,"[time-to-make, course, main-ingredient, cuisin...",[youllneverneedanotherbananabreadrecipeeveraga...,"[butter, granulatedsugar, eggs, bananas, all-p...","[time-to-make, course, main-ingredient, cuisin..."
1,430938,5,famous fat free creamy dill dip,"[weeknight, 15-minutes-or-less, time-to-make, ...",[imadeupthisrecipeafterhavingdilldipatafamilyg...,"[fatfreecreamcheese, nonfatsourcream, dillweed...","[weeknight, 15-minutes-or-less, time-to-make, ..."


In [35]:
rec["rec"] = rec["tags"] + rec["ingredients"]
rec = rec[~rec.duplicated("name")]
rec.reset_index(drop=True,inplace=True)

In [36]:
rec.head(2)

,recipe_id,rating,name,tags,description,ingredients,rec
0,2886,5,best banana bread,"[time-to-make, course, main-ingredient, cuisin...",[youllneverneedanotherbananabreadrecipeeveraga...,"[butter, granulatedsugar, eggs, bananas, all-p...","[time-to-make, course, main-ingredient, cuisin..."
1,430938,5,famous fat free creamy dill dip,"[weeknight, 15-minutes-or-less, time-to-make, ...",[imadeupthisrecipeafterhavingdilldipatafamilyg...,"[fatfreecreamcheese, nonfatsourcream, dillweed...","[weeknight, 15-minutes-or-less, time-to-make, ..."


In [38]:
rec['rec'][0]

['time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'north-american',
 'breads',
 'fruit',
 'american',
 'oven',
 'dietary',
 'quick-breads',
 'equipment',
 '4-hours-or-less',
 'butter',
 'granulatedsugar',
 'eggs',
 'bananas',
 'all-purposeflour',
 'bakingsoda',
 'salt',
 'vanilla']

In [39]:
cv = TfidfVectorizer()
rec_tfidf = cv.fit_transform(rec["rec"])
name_tfidf = cv.fit_transform(rec["name"])
rec_consin_sim = linear_kernel(rec_tfidf,rec_tfidf)
name_consin_sim = linear_kernel(name_tfidf,name_tfidf)

AttributeError: 'list' object has no attribute 'lower'